In [41]:
import cv2
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import compress

import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian,unary_from_softmax

import subprocess

from glob import glob
import os
from tqdm import tqdm
from numpy.lib.stride_tricks import as_strided
import csv
import copy

In [56]:
IMAGES_FOLDER_PATH = "../datasets/custom_still"


# Point Clouds 
# INITIAL_POINT_CLOUD = '../output/initial_point_cloud.ply'
# FINAL_POINT_CLOUD = '../output/final_point_cloud.ply'

# Bundle File
# BUNDLE_FILE = '../output/bundle.out'

# # Shi-Tomasi parameters
# feature_params = dict(maxCorners = 5000, 
#                       qualityLevel = 0.03, 
#                       minDistance = 10, 
#                       blockSize = 15
#                       )

# # Lucas-Kanade parameters
# lk_params = dict(   winSize  = (25,25),
#                     maxLevel = 8,
#                     criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.3))
# Ceres-Solver parameters
CERES_PARAMS = dict(
                    solver = '../ceres-bin/bin/bundle_adjuster',
                    maxIterations = 1000,
                    input_ply = '../output/initial.ply',
                    output_ply = '../output/final.ply',
                    inner_iterations = 'true',
                    nonmonotonic_steps = 'false'
                    )

CAMERA_PARAMS = dict(fx=1781,
                     fy=1781,
                     cx=960,
                     cy=540,
                     k1=0,
                     k2=0,
                     s=0,
                    )
EXTRINSIC_FILE = '../output/extrinsics.csv'



# Optical Flow Plot
# OPTICAL_FLOW_PLOT = '../output/optical_flow.png'

In [55]:
args = dict()
args['min_depth'] = 1
args['max_depth'] = 4
args['patch_radius'] = 1
args['iterations'] = 100
args['std_p'] = (3,3)
args['std_c'] = (20,20,20)
args['wt'] = .5
args['penalty'] = .5
args['folder'] = "custom_still"
args['num'] = 5
args['wta'] = True
# dense_depth(args)
model = DenseReconstruction(args)
model.create_reference()
model.depth_utility()

Doing Plane Sweep Calculation for photoconsistency
Number of depth samples:  5


100%|██████████| 5/5 [00:15<00:00,  3.13s/it]


Finished Plane Sweep Calculation
Dense Map calculation
Using trunc linear
Finished solving


In [54]:
class DenseReconstruction :
    def __init__(self,args):
        self.args = args
        self.folder = args['folder'].split("_")[0]
        self.num_samples = int(args['num'])
        self.wta = args['wta']
        self.min_depth = float(args['min_depth'])
        self.patch_radius = int(args['patch_radius'])
        self.scale = 1
        self.iterations = args['iterations']
        self.max_depth = float(args['max_depth'])
        self.depth_samples = np.zeros(int(args['num']))
        self.ref = cv2.imread(IMAGES_FOLDER_PATH+"/"+self.args['folder']+"_1.jpg")
        self.outfile = '../output/cost_volume_'+str(self.depth_samples.shape[0])+'__'+str(args["std_c"])+'__depth_map.png'
        self.step = (1.0) / (self.num_samples - 1.0)
        self.pc = 0
        self.wt = args['wt']
        self.std_c = args['std_c']
        self.std_p = args['std_p']
        self.max_penalty = args['penalty']
        self.ratio = .55
    def create_reference(self):
        self.ref = cv2.pyrDown(self.ref)
        # Mean shifting image
        self.ref = cv2.pyrMeanShiftFiltering(self.ref, 20, 20, 1)
        self.ref = cv2.cvtColor(self.ref, cv2.COLOR_BGR2Lab)
    def depth_utility(self):
        for val,_ in zip(range(int(args['num'])),range(int(args['num']))):
            sample = (self.max_depth * self.min_depth) / (self.max_depth - (self.max_depth - self.min_depth) * val * self.step)
            self.depth_samples[val] = CAMERA_PARAMS['fx']/sample
        print("Doing Plane Sweep Calculation for photoconsistency")
#         self.pc = self.plane_sweep_util()
        self.plane_sweep_util()
        print("Finished Plane Sweep Calculation")
        print("Dense Map calculation")
        self.Dense_Construction()
        print("Finished solving")
    def plane_sweep_util(self):
        print("Number of depth samples: ",self.depth_samples.shape[0])
        C = []
        R = []
        if(self.wta):
            K = np.array([
            [CAMERA_PARAMS['fx'], CAMERA_PARAMS['s'], CAMERA_PARAMS['cx']],
            [ 0.0,CAMERA_PARAMS['fy'],CAMERA_PARAMS['cy']],
            [ 0.0, 0.0,1.0],])
        
        # Get extrinsics
        with open(EXTRINSIC_FILE) as ext_file:
            csv_reader = csv.reader(ext_file, delimiter=',')
            if(self.wta):
                for row in csv_reader:
                    p = [float(r) for r in row[:-1]]
                    rot, _ = cv2.Rodrigues(np.array(p[:3]))
                    trans = np.array(p[3:6])
                    c = -1 * np.linalg.inv(rot) @ trans
                    R.append(rot)
                    C.append(c)

        # Get all images
        all_img = []
        scaled_gray_images = []
#         print("test2",len(R))
        for file in sorted(os.listdir(IMAGES_FOLDER_PATH))[:len(R)] :  # Get as many images as the extrinsics available
            if file.endswith('.jpg') :
                im = cv2.imread(os.path.join(IMAGES_FOLDER_PATH, file))
                im = im.astype(np.float32)
                all_img.append(im)
#         print("test3",len(all_img))
        for img in all_img :
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_img = cv2.pyrDown(gray_img)
            scaled_gray_images.append(gray_img)
        if(self.wta):
            ref_img = scaled_gray_images[0]
            height, width = ref_img.shape

        num_images = len(all_img)
        cost_volume_arr = np.zeros((self.depth_samples.shape[0], height, width))

        for idx, depth in enumerate(tqdm(self.depth_samples)):

            homographies = np.zeros((num_images, 3, 3))
            warped_images = []

            for ind,_ in zip(range(num_images),range(num_images)) :
                scale = 1
                h = self.HomographyFrom(K, C[0], R[0], C[ind], R[ind], depth)
                actual_scale = 2
                if(self.wta):
                    h[:,:2] = h[:,:2]*actual_scale
                    h[2,:] *= actual_scale
                    homographies[ind,:,:] = h

            for i,_ in zip(range(1, num_images),range(num_images-1)):
                warp = cv2.warpPerspective(scaled_gray_images[i], homographies[i], ref_img.shape[::-1], cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
                warped_images.append(warp)

            if(self.wta):
                ref_img_patches = as_strided(ref_img, shape=(ref_img.shape[0] - 2*self.patch_radius,ref_img.shape[1] - 2*self.patch_radius, 2*self.patch_radius + 1, 2*self.patch_radius + 1),strides=ref_img.strides + ref_img.strides, writeable=False)

            h, w, _, _ = ref_img_patches.shape
            patch_size = 2*self.patch_radius + 1
            patch_radius = copy.deepcopy(self.patch_radius)
            if(patch_radius):
                ref_img_patches = ref_img_patches.reshape((ref_img_patches.shape[0]*ref_img_patches.shape[1], patch_size**2))
                warp_patches = np.zeros((len(warped_images), ref_img_patches.shape[0], ref_img_patches.shape[1]))
            for i,_ in zip(range(len(warped_images)),range(len(warped_images))):
                if(self.wta):
                    x = as_strided(warped_images[i], shape=(warped_images[i].shape[0] - 2*self.patch_radius,
                                    warped_images[i].shape[1] - 2*self.patch_radius, 2*self.patch_radius + 1, 2*self.patch_radius + 1),
                                    strides=warped_images[i].strides + warped_images[i].strides, writeable=False)
                    self.patch_size = copy.deepcopy(patch_size)
                    x = x.reshape((x.shape[0]*x.shape[1], patch_size**2))
                    warp_patches[i,:,:] = x

            L1_diff = self.Sad(ref_img_patches, warp_patches)
            score = self.MergeScores(L1_diff, valid_ratio = 0.5)
            if(self.wta):
                patch_radius = copy.deepcopy(self.patch_radius)
            # Border pixels take values of the neighboring pixels
                cost_volume_arr[idx, self.patch_radius:height-self.patch_radius, self.patch_radius:width-self.patch_radius] = score.reshape((h,w))
                patch_radius = copy.deepcopy(self.patch_radius)
                cost_volume_arr[idx, 0: self.patch_radius, :] = cost_volume_arr[idx, self.patch_radius, :]
                if(self.wta):
                cost_volume_arr[idx, height-patch_radius+1:, :] = cost_volume_arr[idx, height-patch_radius, :]
            
            patch_radius = copy.deepcopy(self.patch_radius)
            cost_volume_arr[idx, :, 0: patch_radius] = cost_volume_arr[idx, :, patch_radius].reshape((cost_volume_arr[idx, :, patch_radius].shape[0],1))
            cost_volume_arr[idx, :, width-patch_radius+1:] = cost_volume_arr[idx, :, width-patch_radius].reshape((cost_volume_arr[idx, :, width-patch_radius].shape[0],1))

        cost_volume_arr = self.Modulate(cost_volume_arr)

        # Saving convention
        #     np.savez_compressed(outfile, pc_cost=cost_volume_arr, dir=folder, max_d=max_depth, min_d=min_depth)

        self.pc =  cost_volume_arr.astype('float32')

    def Sad(self,ref_patch, warp_patch) :
        return np.sum(np.abs(warp_patch - ref_patch), axis=2)

    def HomographyFrom(self,K, C1, R1, C2, R2, dep):

        # C1, R1 : Reference Image
        H  = dep * K @ R2 @ R1.T @ np.linalg.inv(K)
        H[:,2] += K @ R2 @ (C1 - C2)
        return H


    def MergeScores(self,scores):
        '''
        Takes the average of top k values in array. k == valid_scores.
        '''
        self.num_valid_scores = int(scores.shape[0] * self.ratio)
        if(self.wta):
            ix = np.argpartition(scores, self.num_valid_scores, axis=0)
            ix = ix[:self.num_valid_scores,:]
        if(self.wta):
            srt = np.take_along_axis(scores, ix, axis=0)

        return (np.sum(srt, axis=0) / self.num_valid_scores)

    def GetMin(self,values):

        f = s = 0
        f, s = np.partition(values, 1)[0:2]
        return f, s


    def Modulate(self,cost_volume_arr):

        first = 0
        second = 0
        confidence = 0
        num_samples = cost_volume_arr.shape[0]

        for r,_ in zip(range(cost_volume_arr.shape[1]),range(cost_volume_arr.shape[1])):
            for _,c in zip(range(cost_volume_arr.shape[2]),range(cost_volume_arr.shape[2])):
                if(self.wta):
                    values = cost_volume_arr[:, r, c]
                    first, second = self.GetMin(values, num_samples)
                    confidence = (second + 1) / (first + 1)
                    cost_volume_arr[:, r, c] = values * confidence
        return cost_volume_arr
    def Dense_Construction(self):
        labels = self.pc.shape[0]
#         iters = params['iterations']
#         weight = params['wt']
#         pos_std = params['std_p']
#         rgb_std = params['std_p']
#         max_penalty = params['max_penalty']
#         print("test4")
        # Get initial crude depth map from photoconsistency
        if self.wta :
            self.compute_unary_image(self.pc)
#         print("test5")
        # Normalize values for each pixel location
        for r,_ in zip(range(self.pc.shape[1]),range(self.pc.shape[1])):
            for _,c in zip(range(self.pc.shape[2]),range(self.pc.shape[2])):
                if np.sum(self.pc[:, r, c]) <= 1e-9:
                    self.pc[:, r, c] = 0.0
                else:
                    self.pc[:, r, c] = self.pc[:, r, c]/np.sum(self.pc[:, r, c])
#         print("test6")
        # Convert to class probabilities for each pixel location
        unary = unary_from_softmax(self.pc)

        d = dcrf.DenseCRF2D(self.ref.shape[1], self.ref.shape[0], labels)

        # Add photoconsistency score as uanry potential. 16-size vector
        # for each pixel location
        d.setUnaryEnergy(unary)
        # Add color-dependent term, i.e. features are (x,y,r,g,b)
        d.addPairwiseBilateral(sxy=self.std_p, srgb=self.std_c, rgbim=self.ref, compat=np.array([self.wt, labels*self.max_penalty]), kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

        # Run inference steps
        Q = d.inference(self.iterations)

        # Extract depth values. Map to [0-255]
        self.MAP = np.argmax(Q, axis=0).reshape((self.ref.shape[:2]))
        if(self.wta):
            self.depth_map = np.zeros((self.MAP.shape[0], self.MAP.shape[1]))

        for i,_ in zip(range(self.MAP.shape[0]),range(self.MAP.shape[0])):
            for j,_ in zip(range(self.MAP.shape[1]),range(self.MAP.shape[0])):
                self.depth_map[i,j] = self.depth_samples[self.MAP[i,j]]

        min_val = np.min(self.depth_map)
        max_val = np.max(self.depth_map)

        for i in range(self.MAP.shape[0]):
            for j in range(self.MAP.shape[1]):
                self.depth_map[i,j] = ((self.depth_map[i,j] - min_val)/(max_val - min_val)) * (255)

        # Upsampling depth map
        cv2.imwrite(self.outfile, self.depth_map)
    def compute_unary_image(self,unary):
#         print("check",unary.shape,depth_samples.shape)
        gd = np.argmin(unary, axis=0)
        gd_im = np.zeros((unary.shape[1], unary.shape[2]))
        for i,_ in zip(range(unary.shape[1]),range(unary.shape[1])):
            for j,_ in zip(range(unary.shape[2]),range(unary.shape[1])):
                gd_im[i,j] = ((self.depth_samples[gd[i,j]] - np.min(self.depth_samples)) * 255.0) / (np.max(self.depth_samples) - np.min(self.depth_samples))
#         cv2.imwrite(outfile, gd_im)